In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

import sys
sys.path.append("..")
from model import lstm,cnn

# 数据统计

In [75]:
import preprocess

base_path = r"F:\mystyle\git\Sentiment-Analysis\data\ChnSentiCorp_htl_ba_2000"
data, label = preprocess.read_data(base_path)

In [76]:
print("Data Count : {0} ,Label Count : {1}".format(len(data),len(label)))
emotion = dict()
for v in label:
    emotion.setdefault("neg", 0)
    emotion.setdefault("pos", 0)
    emotion[v] += 1
    
    
print("Neg Count : {}".format(emotion["neg"]))
print("Pos Count : {}".format(emotion["pos"]))

Data Count : 1953 ,Label Count : 1953
Neg Count : 964
Pos Count : 989


# 数据预处理

## 替换\n\t

In [24]:
data[0]

'标准间太差 房间还不如3星的 而且设施非常陈旧.建议酒店把老的标准间从新改善.\n\n\n\n'

In [77]:
for i,doc in enumerate(data):
    doc = doc.replace("\n"," ")
    doc = doc.replace("\t"," ")
    data[i] = doc

In [30]:
data[0]

'标准间太差 房间还不如3星的 而且设施非常陈旧.建议酒店把老的标准间从新改善.    '

## 去掉标点符号

In [78]:
import re
from zhon.hanzi import punctuation as ch_p
from string import punctuation as en_p

for i,doc in enumerate(data):
    doc = re.sub(r"[{0}{1}]+".format(ch_p, en_p)," ", doc)
    doc = doc.strip()
    data[i] = doc

In [47]:
data[0]

'标准间太差 房间还不如3星的 而且设施非常陈旧 建议酒店把老的标准间从新改善'

## 分词

In [79]:
import jieba

for i,doc in enumerate(data):
    cut_doc = jieba.cut(doc)
    cut_doc = " ".join(cut_doc)
    data[i] = cut_doc


In [57]:
data[0]

'标准间 太 差   房间 还 不如 3 星 的   而且 设施 非常 陈旧   建议 酒店 把 老 的 标准间 从 新 改善'

## 去掉停用词

In [80]:
filename = r"F:\mystyle\git\Sentiment-Analysis\data\stopWord.txt"
stop_words = list()
with open(filename,"r",encoding="utf-8") as f:
    for line in f:
        line = line.replace("\n", "")
        stop_words.append(line)

In [81]:
for i,doc in enumerate(data):
    words = doc.split(" ")
    temp = []
    for word in words:
        if word in stop_words:
            continue
        temp.append(word)
    data[i] = " ".join(temp)

In [82]:
data[0]

'标准间 太 差 房间 星 设施 陈旧 建议 酒店 标准间 新 改善'

# 识别

## TF-IDF + RF

### TFIDF

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(lowercase=False, max_features = 200)
tfidf_matrix = tfidf_v.fit_transform(data)

### 分离训练集和验证集

In [101]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix,label,test_size = 0.1)

print("Train Count : {}".format(X_train.shape[0]))
print("Test Count : {}".format(X_test.shape[0]))

Train Count : 1757
Test Count : 196


### RF

In [114]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = 150
max_features = "sqrt"

rfc = RandomForestClassifier(n_estimators=n_estimators,max_features=max_features)
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [115]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true = rfc.predict(X_test),y_pred = y_test)

0.8877551020408163

## RNN

## CNN